# Detection and Classification of Malicious URLs

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Load the Data

In [3]:
url = 'D:/Google Drive/Colab Notebooks/SCS_3253_047_Machine_Learning/Term_Project/datasets/ISCX-URL2016_All.csv'
df = pd.read_csv(url)
df_backup = df.copy()

y_feature = 'URL_Type_obf_Type'

### Explore the data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36707 entries, 0 to 36706
Data columns (total 80 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Querylength                      36707 non-null  int64  
 1   domain_token_count               36707 non-null  int64  
 2   path_token_count                 36707 non-null  int64  
 3   avgdomaintokenlen                36707 non-null  float64
 4   longdomaintokenlen               36707 non-null  int64  
 5   avgpathtokenlen                  36427 non-null  float64
 6   tld                              36707 non-null  int64  
 7   charcompvowels                   36707 non-null  int64  
 8   charcompace                      36707 non-null  int64  
 9   ldl_url                          36707 non-null  int64  
 10  ldl_domain                       36707 non-null  int64  
 11  ldl_path                         36707 non-null  int64  
 12  ldl_filename      

In [5]:
df.describe()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath
count,36707.000000,36707.000000,36707.000000,36707.000000,36707.000000,36427.000000,36707.000000,36707.000000,36707.000000,36707.000000,...,36707.000000,36707.000000,36707.000000,36707.000000,36707.000000,36707.000000,28239.000000,36471.000000,36667.000000,36701.000000
mean,21.908110,2.857439,8.954668,5.401719,9.746016,4.556261,2.857439,14.976517,10.216117,3.363909,...,1.727545,3.600049,2.685237,1.903261,0.722350,0.842443,0.677459,0.715834,0.508212,-0.208900
std,94.280669,0.906562,4.278406,1.986858,4.856580,2.839388,0.906562,13.351315,10.845807,13.447289,...,2.719231,4.783213,4.557171,4.512421,0.048820,0.066251,0.450683,0.411712,0.519695,0.891581
min,0.000000,2.000000,0.000000,1.500000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,0.419560,0.561913,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,2.000000,6.000000,4.000000,6.000000,3.000000,2.000000,7.000000,5.000000,0.000000,...,-1.000000,0.000000,0.000000,-1.000000,0.691929,0.795234,0.731621,0.719904,0.000000,-1.000000
50%,0.000000,3.000000,8.000000,5.000000,9.000000,4.100000,3.000000,13.000000,8.000000,0.000000,...,1.000000,1.000000,0.000000,-1.000000,0.723003,0.835752,0.790535,0.785150,0.729292,-1.000000
75%,19.000000,3.000000,11.000000,6.500000,12.000000,5.000000,3.000000,19.000000,12.000000,1.000000,...,3.000000,6.000000,4.000000,3.000000,0.754305,0.884735,0.851410,0.884662,0.910005,0.758695
max,1385.000000,19.000000,68.000000,29.500000,63.000000,105.000000,19.000000,193.000000,142.000000,207.000000,...,24.000000,40.000000,39.000000,40.000000,0.895403,1.000000,0.962479,1.000000,1.000000,1.000000


In [6]:
df.columns

Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

In [7]:
df.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


In [8]:
print(df[y_feature].value_counts())

Defacement    7930
benign        7781
phishing      7586
malware       6712
spam          6698
Name: URL_Type_obf_Type, dtype: int64


In [9]:
df.dropna(axis='index', inplace=True) # TODO try applying my previous approach if that will help in order or not but based on the current result this is better

In [10]:
print(df[y_feature].value_counts())

spam          5342
malware       4440
phishing      4014
benign        2709
Defacement    2477
Name: URL_Type_obf_Type, dtype: int64


In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(df.loc[:, df.columns != y_feature]), columns=df.columns[:-1] )

y = df[y_feature]

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()    
encoder.fit(y)

y = encoder.transform(y)

In [12]:
print(X.shape)
print(y.shape)

(18982, 79)
(18982,)


In [13]:
# I initially use this approach:
# https://towardsdatascience.com/feature-selection-correlation-and-p-value-da8921bfb3cf

# However, I found that chi2 is a better approach:
# https://towardsdatascience.com/chi-square-test-for-feature-selection-in-machine-learning-206b1f0b8223

from sklearn.feature_selection import chi2

chi_scores = chi2(X, y)
p_values = pd.Series(chi_scores[1], index=X.columns)
p_values.sort_values(ascending=False, inplace=True)
p_values.index

Index(['Entropy_URL', 'Path_LongestWordLength', 'dld_domain',
       'sub-Directory_LongestWordLength', 'isPortEighty', 'avgpathtokenlen',
       'host_letter_count', 'Entropy_Domain', 'domainlength', 'ldl_domain',
       'longdomaintokenlen', 'URL_sensitiveWord', 'Directory_LetterCount',
       'Directory_DigitCount', 'NumberofDotsinURL', 'Domain_LongestWordLength',
       'delimeter_Domain', 'NumberRate_DirectoryName', 'File_name_DigitCount',
       'path_token_count', 'avgdomaintokenlen', 'SymbolCount_Directoryname',
       'ldl_filename', 'dld_filename', 'Entropy_Extension',
       'Filename_LetterCount', 'pathurlRatio', 'executable', 'spcharUrl',
       'CharacterContinuityRate', 'charcompvowels',
       'Arguments_LongestWordLength', 'delimeter_path',
       'Entropy_DirectoryName', 'Entropy_Filename', 'SymbolCount_URL',
       'NumberRate_FileName', 'NumberRate_Extension', 'domainUrlRatio',
       'SymbolCount_FileName', 'pathDomainRatio', 'URL_Letter_Count',
       'SymbolCount

In [14]:
# top 10 most significant features using chi score. note using only the top features reduce the accuracy score of the models
p_values.head(20)

Entropy_URL                        1.272731e-01
Path_LongestWordLength             1.380409e-02
dld_domain                         4.124519e-04
sub-Directory_LongestWordLength    3.440676e-04
isPortEighty                       2.881362e-05
avgpathtokenlen                    1.192005e-07
host_letter_count                  1.007751e-08
Entropy_Domain                     6.009946e-09
domainlength                       8.353102e-10
ldl_domain                         7.808014e-11
longdomaintokenlen                 6.682376e-11
URL_sensitiveWord                  1.072193e-11
Directory_LetterCount              3.687000e-14
Directory_DigitCount               6.825812e-17
NumberofDotsinURL                  6.505166e-18
Domain_LongestWordLength           5.178927e-19
delimeter_Domain                   9.376033e-20
NumberRate_DirectoryName           5.496251e-21
File_name_DigitCount               2.638187e-21
path_token_count                   2.532674e-21
dtype: float64

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# From machine learning cheat sheet
# Speed
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Accuracy
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score

random_state=42

max_depth=10

dec_tree_clf = DecisionTreeClassifier(random_state=random_state, max_depth=max_depth)
log_clf = LogisticRegression(random_state=random_state, max_iter=1000)

svm_lin_clf = SVC(kernel='linear')
svm_rbf_clf = SVC(kernel='rbf')
rnd_clf = RandomForestClassifier(random_state=random_state, max_depth=max_depth)
ada_clf = AdaBoostClassifier(random_state=random_state)
gra_clf = GradientBoostingClassifier(random_state=random_state, max_depth=max_depth)

classifiers = {
    'DecisionTreeClassifier': dec_tree_clf,
    'LogisticRegression': log_clf,    
    'SVM_linear': svm_lin_clf,
    'SVM_rbf': svm_rbf_clf,
    'RandomForestClassifier': rnd_clf,
    'AdaBoostClassifier': ada_clf,
    'GradientBoostingClassifier': gra_clf
}

In [16]:
%%time
print("Running the models without optimization:")
for key in classifiers.keys():
    print('---------------------------------------------------------------------------')

    clf = classifiers[key]    

    %timeit clf.fit(X_train, y_train)

    y_pred = clf.predict(X_train)
        
    print(key, ': ', accuracy_score(y_train, y_pred))
    

Running the models without optimization:
---------------------------------------------------------------------------
271 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
DecisionTreeClassifier :  0.9180770497201185
---------------------------------------------------------------------------
3.62 s ± 184 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
LogisticRegression :  0.8227856437273625
---------------------------------------------------------------------------
9.67 s ± 313 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
SVM_linear :  0.8491274283832729
---------------------------------------------------------------------------
11.9 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
SVM_rbf :  0.8715179453407969
---------------------------------------------------------------------------
2.27 s ± 43.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
RandomForestClassifier :  0.9599604873230161
------------------------------------------------

In [17]:
import multiprocessing 
multiprocessing.cpu_count()

8